# IMPORTANT!!!
> Move out of bak, otherwise can't read the env_vars.env file as is.
>
> 

Notebook requirements and how start YugabyteDB locally (mac/linux) with yb-ctl

### Create the notebook variables 

> IMPORTANT!
> 
> Do NOT skip editing and running this cell. 
> 


The following Python cell creates and stores the Jupyter notebook variables.
- First, edit the script to select how to assign a value to `MY_SUDO`.
- To run the script, select Execute Cell (Play Arrow in the cell side bar).
- Verify the accuracy of the output values

In [ ]:
!pip install --upgrade pip
!pip install ipython-sql~=0.5 --upgrade
!pip install psycopg2-binary
!pip install SQLAlchemy --upgrade 
!pip install sqlparse
!pip install html2text
!pip install jq

In [1]:
# Env variables for Notebook
import os

env_vars = !cat env_vars.env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value
 
### BEGIN: Edit your MY_SUDO

# Leave it blank forcing a strategy for notebook execution
# MY_SUDO=''

# Specify your sudo password in the cell
#MY_SUDO='password'

# env_vars defines ENV_SUDO='psswrd'
MY_SUDO=os.environ.get('MY_SUDO')


# bash_profSile exports the variable, e.g., export GLOBAL_SUDO='psswrd' 
MY_SUDO=os.environ.get('GLOBAL_SUDO')

### END: MY_SUDO


# Create and assign notebook variables
# env_vars defines the following
MY_DB_NAME=os.environ.get('MY_DB_NAME')
# MY_YB_PATH=os.environ.get('MY_YB_PATH')
MY_YB_PATH=os.environ.get('MY_YB_PATH_LOCAL')
MY_GITPOD_WORKSPACE_URL=os.environ.get('MY_GITPOD_WORKSPACE_URL_LOCAL')

MY_HOST_IPv4_01=os.environ.get('MY_HOST_IPv4_01')
MY_HOST_IPv4_02=os.environ.get('MY_HOST_IPv4_02')
MY_HOST_IPv4_03=os.environ.get('MY_HOST_IPv4_03')
MY_TSERVER_WEBSERVER_PORT=os.environ.get('MY_TSERVER_WEBSERVER_PORT')
MY_DATA_DDL_FILE_1=os.environ.get("MY_DATA_DDL_FILE_1")
MY_DATA_DML_FILE_1=os.environ.get("MY_DATA_DML_FILE_1")
MY_UTIL_FUNCTIONS_FILE=os.environ.get("MY_UTIL_FUNCTIONS_FILE")
MY_UTIL_YBTSERVER_METRICS_FILE=os.environ.get("MY_UTIL_YBTSERVER_METRICS_FILE")
# Current directory of project and related child folders

MY_NOTEBOOK_DIR=os.getcwd()
MY_NOTEBOOK_DATA_FOLDER=MY_NOTEBOOK_DIR +'/data'

MY_NOTEBOOK_UTILS_FOLDER=MY_NOTEBOOK_DIR + '/utils'

# Store the note book values for other notebooks to use
%store MY_SUDO
%store MY_DB_NAME
%store MY_YB_PATH
%store MY_HOST_IPv4_01
%store MY_HOST_IPv4_02
%store MY_HOST_IPv4_03
%store MY_NOTEBOOK_DIR
%store MY_TSERVER_WEBSERVER_PORT
%store MY_NOTEBOOK_DATA_FOLDER
%store MY_NOTEBOOK_UTILS_FOLDER
%store MY_DATA_DDL_FILE_1
%store MY_DATA_DML_FILE_1
%store MY_UTIL_FUNCTIONS_FILE
%store MY_UTIL_YBTSERVER_METRICS_FILE

Stored 'MY_SUDO' (str)
Stored 'MY_DB_NAME' (str)
Stored 'MY_YB_PATH' (str)
Stored 'MY_HOST_IPv4_01' (str)
Stored 'MY_HOST_IPv4_02' (str)
Stored 'MY_HOST_IPv4_03' (str)
Stored 'MY_NOTEBOOK_DIR' (str)
Stored 'MY_TSERVER_WEBSERVER_PORT' (str)
Stored 'MY_NOTEBOOK_DATA_FOLDER' (str)
Stored 'MY_NOTEBOOK_UTILS_FOLDER' (str)
Stored 'MY_DATA_DDL_FILE_1' (str)
Stored 'MY_DATA_DML_FILE_1' (str)
Stored 'MY_UTIL_FUNCTIONS_FILE' (str)
Stored 'MY_UTIL_YBTSERVER_METRICS_FILE' (str)


### Create the loopback host aliases
> Important!
> 
> The script requires the password for `sudo`.
> 
> The bash cell utilizes the notebook variable, `MY_SUDO`.
> 
> If you did not define the `MY_SUDO` notebook variable using one of the suggested options, you can copy this script into the terminal, which is interactive, and manually supply the password.

The YugabyteDB local cluster for this lab requires that your create loop back aliases for your local host (127.0.0.1). The bash cell does the following:
- Deletes existing aliases
- Creates the loopback aliases from `127.0.0.2` to `127.0.0.7`

In [2]:
%%bash -s "$MY_SUDO"  # ifconfig aliases
MY_SUDO=${1}

if ifconfig lo0 | grep 127.0.0.[2-7] > /dev/null
then
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.2
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.3
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.4
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.5
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.6
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.7
fi

echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.2
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.3
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.4
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.5
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.6
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.7

echo ${MY_SUDO} | sudo ifconfig lo0

lo0: flags=8049<UP,LOOPBACK,RUNNING,MULTICAST> mtu 16384
	options=1203<RXCSUM,TXCSUM,TXSTATUS,SW_TIMESTAMP>
	inet 127.0.0.1 netmask 0xff000000 
	inet6 ::1 prefixlen 128 
	inet6 fe80::1%lo0 prefixlen 64 scopeid 0x1 
	inet 127.0.0.2 netmask 0xff000000 
	inet 127.0.0.3 netmask 0xff000000 
	inet 127.0.0.4 netmask 0xff000000 
	inet 127.0.0.5 netmask 0xff000000 
	inet 127.0.0.6 netmask 0xff000000 
	inet 127.0.0.7 netmask 0xff000000 
	nd6 options=201<PERFORMNUD,DAD>


Password:

### Create a YugabyteDB local cluster
- If a YugabyteDB local cluster is running, it will be stopped and destroyed using `yb-ctl`
- To avoid the port 9000 conflict, specifies port `TSERVER_WEBSERVER_PORT` as `8200`

In [3]:
%%bash -s "$MY_YB_PATH" "$MY_TSERVER_WEBSERVER_PORT"  # yb-ctl create
YB_PATH=${1}
TSERVER_WEBSERVER_PORT=${2}

cd $YB_PATH

### Grep port 9000 for conflict
# lsof -nP -iTCP -sTCP:LISTEN | grep 9000

# Stop running cluster
if  pgrep -x "yb-tserver" > /dev/null 
then
    ./bin/yb-ctl stop
    sleep 1
fi

# Destroy cluster
if echo `./bin/yb-ctl status` | grep "Node Count"  > /dev/null 
then
    ./bin/yb-ctl destroy
    sleep 1
fi

# Create cluster
./bin/yb-ctl --rf 3 create  \
--tserver_flags "yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1,ysql_beta_features=true,webserver_port="${TSERVER_WEBSERVER_PORT}  \
--master_flags "yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1" \
--num_shards_per_tserver=1  \
--placement_info "cloud1.region1.zone1,cloud2.region2.zone2,cloud3.region3.zone3" 

## add node
## ./bin/yb-ctl add_node --placement_info "cloud1.region1.zone1" 

# Output status
./bin/yb-ctl status

Destroying cluster.
Creating cluster.
Waiting for cluster to be ready.
----------------------------------------------------------------------------------------------------
| Node Count: 3 | Replication Factor: 3                                                            |
----------------------------------------------------------------------------------------------------
| JDBC                : jdbc:postgresql://127.0.0.1:5433/yugabyte                                  |
| YSQL Shell          : bin/ysqlsh                                                                 |
| YCQL Shell          : bin/ycqlsh                                                                 |
| YEDIS Shell         : bin/redis-cli                                                              |
| Web UI              : http://127.0.0.1:7000/                                                     |
| Cluster Data        : /Users/seth/yugabyte-data                                                  |
--------------------

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_TSERVER_WEBSERVER_PORT"  # yb-ctl create
YB_PATH=${1}
TSERVER_WEBSERVER_PORT=${2}

cd $YB_PATH


# Output status
./bin/yb-ctl status

### YCQL API

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_TSERVER_WEBSERVER_PORT"  # yb-ctl create
YB_PATH=${1}
TSERVER_WEBSERVER_PORT=${2}

cd $YB_PATH


# Output status
./bin/ycqlsh -e "
  select cql_version from system.local;
"  

### YSQL API

In [8]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # create database
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

# drop and create
./bin/ysqlsh -d yugabyte -c "drop database if exists "${DB_NAME}";"  
./bin/ysqlsh -d yugabyte -c "create database "${DB_NAME}";" 

# list dbs
./bin/ysqlsh -d yugabyte -c "\l"

                                   List of databases
      Name       |  Owner   | Encoding | Collate |    Ctype    |   Access privileges   
-----------------+----------+----------+---------+-------------+-----------------------
 db_ybu          | yugabyte | UTF8     | C       | en_US.UTF-8 | 
 postgres        | postgres | UTF8     | C       | en_US.UTF-8 | 
 system_platform | postgres | UTF8     | C       | en_US.UTF-8 | 
 template0       | postgres | UTF8     | C       | en_US.UTF-8 | =c/postgres          +
                 |          |          |         |             | postgres=CTc/postgres
 template1       | postgres | UTF8     | C       | en_US.UTF-8 | =c/postgres          +
                 |          |          |         |             | postgres=CTc/postgres
 yugabyte        | postgres | UTF8     | C       | en_US.UTF-8 | 
(6 rows)



ERROR:  database "db_ybu" is being accessed by other users
DETAIL:  There are 2 other sessions using the database.
ERROR:  database "db_ybu" already exists
